# Unfairness Mitigation with Fairlearn and Azure Machine Learning
**This notebook shows how to upload results from Fairlearn's GridSearch mitigation algorithm into a dashboard in Azure Machine Learning Studio**
## Disclaimer

By accessing this code, you acknowledge the code is made available for presentation and demonstration purposes only and that the code: (1) is not subject to SOC 1 and SOC 2 compliance audits; (2) is not designed or intended to be a substitute for the professional advice, diagnosis, treatment, or judgment of a certified financial services professional; (3) is not designed, intended or made available as a medical device; and (4) is not designed or intended to be a substitute for professional medical advice, diagnosis, treatment or judgement. Do not use this code to replace, substitute, or provide professional financial advice or judgment, or to replace, substitute or provide medical advice, diagnosis, treatment or judgement. You are solely responsible for ensuring the regulatory, legal, and/or contractual compliance of any use of the code, including obtaining any authorizations or consents, and any solution you choose to build that incorporates this code in whole or in part.

© 2021 Microsoft Corporation. All rights reserved
## Table of Contents

1. Introduction
1. Loading the Data
1. Training an Unmitigated Model)
1. Mitigation with GridSearch
1. Uploading a Fairness Dashboard to Azure
    1. Registering models
    1. Computing Fairness Metrics
    1. Uploading to Azure
1. Conclusion

<a id="Introduction"></a>
## Introduction
This notebook shows how to use Fairlearn (an open-source fairness assessment and unfairness mitigation package) and Azure Machine Learning Studio for a binary classification problem. In this notebook, we will be using a loan decision dataset. The label indicates whether or not each individual repaid a loan in the past. We will use the data to train a predictor to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.

 

Farilearn mitigates disparity using two types of algorithms, reduction, and post-processing. We will apply the grid search algorithm, a reduction algorithm, from the Fairlearn package using a specific notion of fairness called Demographic Parity. Reduction algorithms reduce disparity by training models on reweighted datasets. This produces a set of models, and we will view these in a dashboard both locally and in the Azure Machine Learning Studio. We will also see the trade-off between model performance and disparity visually through Fairlearn dashboards, which can help make an informed decision about the model.

### Setup

To use this notebook, an Azure Machine Learning workspace is required. This notebook also requires the following packages:
* `azureml-contrib-fairness`
* `fairlearn==0.4.6` (v0.5.0 will work with minor modifications)
* `joblib`
* `liac-arff`

Fairlearn relies on features introduced in v0.22.1 of `scikit-learn`. If you have an older version already installed, please uncomment and run the following cell:

In [1]:
!pip install -U sklearn
!pip install fairlearn
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

Requirement already up-to-date: sklearn in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (0.0)
The scikit-learn version is 0.24.2.


Finally, please ensure that when you downloaded this notebook, you also downloaded the `fairness_nb_utils.py` file from the same location, and placed it in the same directory as this notebook.

<a id="LoadingData"></a>
## Loading the Data
We use the well-known `adult` loan dataset, which we will fetch from the OpenML website. We start with a fairly unremarkable set of imports:

In [22]:
from fairlearn.reductions import GridSearch, DemographicParity, ErrorRate
#from fairlearn.widget import FairlearnDashboard

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import Pipeline

import pandas as pd

ModuleNotFoundError: No module named 'fairlearn.widget'

We can now load and inspect the data:

In [3]:
from sklearn.datasets import fetch_openml

data = fetch_openml(data_id=1590, as_frame=True)
# Extract the items we want
X_raw = data.data
y = (data.target == '>50K') * 1

print(X_raw["race"].value_counts())
X_raw

White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States
48838,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States
48839,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States
48840,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States


In [4]:
dfupdate=X_raw.sample(5000)  #Number of sample you want to update

dfupdate.race="Black" #Assigning new category

X_raw.update(dfupdate)
#update_list = dfupdate.index.tolist()
print(X_raw["race"].value_counts())

for col in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country' ]:
    X_raw[col] = X_raw[col].astype('category')
X_raw.dtypes
print(X_raw)

White                 37483
Black                  9211
Asian-Pac-Islander     1373
Amer-Indian-Eskimo      420
Other                   355
Name: race, dtype: int64
        age     workclass    fnlwgt     education  education-num  \
0      25.0       Private  226802.0          11th            7.0   
1      38.0       Private   89814.0       HS-grad            9.0   
2      28.0     Local-gov  336951.0    Assoc-acdm           12.0   
3      44.0       Private  160323.0  Some-college           10.0   
4      18.0           NaN  103497.0  Some-college           10.0   
...     ...           ...       ...           ...            ...   
48837  27.0       Private  257302.0    Assoc-acdm           12.0   
48838  40.0       Private  154374.0       HS-grad            9.0   
48839  58.0       Private  151910.0       HS-grad            9.0   
48840  22.0       Private  201490.0       HS-grad            9.0   
48841  52.0  Self-emp-inc  287927.0       HS-grad            9.0   

           marital

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


We are going to treat the sex and race of each individual as protected attributes, and in this particular case we are going to remove these attributes from the main data. Protected attributes are often denoted by 'A' in the literature, and we follow that convention here:

In [5]:
A = X_raw[['sex','race']]
X_raw = X_raw.drop(labels=['sex', 'race'], axis = 1)

We now preprocess our data. To avoid the problem of data leakage, we split our data into training and test sets before performing any other transformations. Subsequent transformations (such as scalings) will be fit to the training data set, and then applied to the test dataset.

In [6]:
(X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
    X_raw, y, A, test_size=0.3, random_state=12345, stratify=y)

# Ensure indices are aligned between X, y and A,
# after all the slicing and splitting of DataFrames
# and Series

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

We have two types of column in the dataset - categorical columns which will need to be one-hot encoded, and numeric ones which will need to be rescaled. We also need to take care of missing values. We use a simple approach here, but please bear in mind that this is another way that bias could be introduced (especially if one subgroup tends to have more missing values).

For this preprocessing, we make use of `Pipeline` objects from `sklearn`:

In [7]:
numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

Now, the preprocessing pipeline is defined, we can run it on our training data, and apply the generated transform to our test data:

In [8]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

<a id="UnmitigatedModel"></a>
## Training an Unmitigated Model

So we have a point of comparison, we first train a model (specifically, logistic regression from scikit-learn) on the raw data, without applying any mitigation algorithm:

In [9]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

We can view this model in the fairness dashboard, and see the disparities which appear:

<a id="AzureUpload"></a>
## Uploading a Fairness Dashboard to Azure (one time job)

Uploading a fairness dashboard to Azure is a two stage process. The `FairlearnDashboard` invoked in the previous section relies on the underlying Python kernel to compute metrics on demand. This is obviously not available when the fairness dashboard is rendered in AzureML Studio. By default, the dashboard in Azure Machine Learning Studio also requires the models to be registered. The required stages are therefore:
1. Register the dominant models
1. Precompute all the required metrics
1. Upload to Azure

Before that, we need to connect to Azure Machine Learning Studio:

In [10]:
from azureml.core import Workspace, Experiment, Model
import joblib
import os

ws = Workspace.from_config()
ws.get_details()

os.makedirs('models', exist_ok=True)

# Function to register models into Azure Machine Learning
def register_model(name, model):
    print("Registering ", name)
    model_path = "models/{0}.pkl".format(name)
    joblib.dump(value=model, filename=model_path)
    registered_model = Model.register(model_path=model_path,
                                    model_name=name,
                                    workspace=ws)
    print("Registered ", registered_model.id)
    return registered_model.id

# Call the register_model function 
lr_reg_id = register_model("unfairness_model_for_race_v2", unmitigated_predictor)
#  Create a dictionary of model(s) you want to assess for fairness 
sf = { 'Race': A_test.race}
ys_pred = { lr_reg_id:unmitigated_predictor.predict(X_test) }
from fairlearn.metrics._group_metric_set import _create_group_metric_set

dash_dict = _create_group_metric_set(y_true=y_test,
                                    predictions=ys_pred,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')

from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

exp = Experiment(ws, "Unfairness_Experiment_for_Race_v2")
print(exp)

run = exp.start_logging()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness insights of Regression Classifier for Race"
    # Set validate_model_ids parameter of upload_dashboard_dictionary to False if you have not registered your model(s)
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it back and ensure it contains the right information
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
finally:
    run.complete()

Registering  unfairness_model_for_race_v2
Registering model unfairness_model_for_race_v2
Registered  unfairness_model_for_race_v2:2
Experiment(Name: Unfairness_Experiment_for_Race_v2,
Workspace: amlws-sufse2t562jqy-pm0508)

Uploaded to id: 6c5bbbac-0d00-4153-bb84-09a8dad5966a



INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_dashboard_validation.py:Starting validation of dashboard dictionary
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_dashboard_validation.py:Validation of dashboard dictionary successful
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Validating model ids exist
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Checking unfairness_model_for_race_v2:2
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Validation of model ids complete
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_fairness_client.py:Uploading y_true
INFO:azureml.FairnessArtifactClient:Uploading to azureml.fairness/dashboard.metrics/6c5bbbac-0d00-4153-bb84-09a8dad5966a/y_true/433111cb-7

<a id="Mitigation"></a>
## Mitigation with GridSearch

The `GridSearch` class in `Fairlearn` implements a simplified version of the exponentiated gradient reduction. The user supplies a standard ML estimator, which is treated as a blackbox - for this simple example, we shall use the logistic regression estimator from scikit-learn. `GridSearch` works by generating a sequence of relabellings and reweightings, and trains a predictor for each.

For this example, we specify demographic parity (on the protected attribute of race) as the fairness metric. Demographic parity requires that individuals are offered the opportunity (a loan in this example) independent of membership in the protected class (i.e., females and males should be offered loans at the same rate). *We are using this metric for the sake of simplicity* in this example; the appropriate fairness metric can only be selected after *careful examination of the broader context* in which the model is to be used.
    

In [11]:
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=71)

With our estimator created, we can fit it to the data. After `fit()` completes, we extract the full set of predictors from the `GridSearch` object.

The following cell trains a many copies of the underlying estimator, and may take a minute or two to run:

In [12]:
sweep.fit(X_train, y_train,
          sensitive_features=A_train.sex)

# For Fairlearn v0.5.0, need sweep.predictors_
predictors = sweep.predictors_

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprec

We could load these predictors into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the predictors which are dominated in the error-disparity space by others from the sweep (note that the disparity will only be calculated for the protected attribute; other potentially protected attributes will *not* be mitigated). In general, one might not want to do this, since there may be other considerations beyond the strict optimisation of error and disparity (of the given protected attribute).

In [13]:
errors, disparities = [], []
for m in predictors:
    classifier = lambda X: m.predict(X)
    
    error = ErrorRate()
    error.load_data(X_train, pd.Series(y_train), sensitive_features=A_train.sex)
    disparity = DemographicParity()
    disparity.load_data(X_train, pd.Series(y_train), sensitive_features=A_train.sex)
    
    errors.append(error.gamma(classifier)[0])
    disparities.append(disparity.gamma(classifier).max())
    
all_results = pd.DataFrame( {"predictor": predictors, "error": errors, "disparity": disparities})

dominant_models_dict = dict()
base_name_format = "census_gs_model_{0}"
row_id = 0
for row in all_results.itertuples():
    model_name = base_name_format.format(row_id)
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["disparity"]<=row.disparity]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        dominant_models_dict[model_name] = row.predictor
    row_id = row_id + 1

We can construct predictions for the dominant models (we include the unmitigated predictor as well, for comparison):

In [14]:
predictions_dominant = {"census_unmitigated": unmitigated_predictor.predict(X_test)}
models_dominant = {"census_unmitigated": unmitigated_predictor}
for name, predictor in dominant_models_dict.items():
    value = predictor.predict(X_test)
    predictions_dominant[name] = value
    models_dominant[name] = predictor

These predictions may then be viewed in the fairness dashboard. We include the race column from the dataset, as an alternative basis for assessing the models. However, since we have not based our mitigation on it, the variation in the models with respect to race can be large.

When using sex as the sensitive feature and accuracy as the metric, we see a Pareto front forming - the set of predictors which represent optimal tradeoffs between accuracy and disparity in predictions. In the ideal case, we would have a predictor at (1,0) - perfectly accurate and without any unfairness under demographic parity (with respect to the protected attribute "sex"). The Pareto front represents the closest we can come to this ideal based on our data and choice of estimator. Note the range of the axes - the disparity axis covers more values than the accuracy, so we can reduce disparity substantially for a small loss in accuracy. Finally, we also see that the unmitigated model is towards the top right of the plot, with high accuracy, but worst disparity.

By clicking on individual models on the plot, we can inspect their metrics for disparity and accuracy in greater detail. In a real example, we would then pick the model which represented the best trade-off between accuracy and disparity given the relevant business constraints.

<a id="AzureUpload"></a>
## Uploading a Fairness Dashboard to Azure (one time job)

Uploading a fairness dashboard to Azure is a two stage process. The `FairlearnDashboard` invoked in the previous section relies on the underlying Python kernel to compute metrics on demand. This is obviously not available when the fairness dashboard is rendered in AzureML Studio. By default, the dashboard in Azure Machine Learning Studio also requires the models to be registered. The required stages are therefore:
1. Register the dominant models
1. Precompute all the required metrics
1. Upload to Azure

Before that, we need to connect to Azure Machine Learning Studio:

In [15]:
from azureml.core import Workspace, Experiment, Model

ws = Workspace.from_config()
ws.get_details()

INFO:azureml.core.workspace:Found the config file in: /config.json


{'id': '/subscriptions/3f01ab49-a56f-4ee7-97fa-d23155156b42/resourceGroups/media-test/providers/Microsoft.MachineLearningServices/workspaces/amlws-sufse2t562jqy-pm0508',
 'name': 'amlws-sufse2t562jqy-pm0508',
 'identity': {'principal_id': 'b65f34f9-0b69-4071-8136-f5b5eb9ba2ed',
  'tenant_id': 'f94768c8-8714-4abe-8e2d-37a64b18216a',
  'type': 'SystemAssigned'},
 'location': 'westus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '43a51ee5-20a5-46a8-ad4b-030b3ad5d13e',
 'sdkTelemetryAppInsightsKey': '19f24253-9564-406c-9a1e-a48a21b145aa',
 'description': '',
 'friendlyName': 'amlws-sufse2t562jqy-pm0508',
 'creationTime': '2021-08-05T05:40:36.5152006+00:00',
 'keyVault': '/subscriptions/3f01ab49-a56f-4ee7-97fa-d23155156b42/resourcegroups/media-test/providers/microsoft.keyvault/vaults/amlwssufkeyvault6fbb802e',
 'applicationInsights': '/subscriptions/3f01ab49-a56f-4ee7-97fa-d23155156b42/resourcegroups/media-test/providers/microsoft.

<a id="RegisterModels"></a>
### Registering Models

The fairness dashboard is designed to integrate with registered models, so we need to do this for the models we want in the Studio portal. The assumption is that the names of the models specified in the dashboard dictionary correspond to the `id`s (i.e. `<name>:<version>` pairs) of registered models in the workspace. We register each of the models in the `models_dominant` dictionary into the workspace. For this, we have to save each model to a file, and then register that file:

In [16]:
import joblib
import os

os.makedirs('models', exist_ok=True)
def register_model(name, model):
    print("Registering ", name)
    model_path = "models/{0}.pkl".format(name)
    joblib.dump(value=model, filename=model_path)
    registered_model = Model.register(model_path=model_path,
                                      model_name=name,
                                      workspace=ws)
    print("Registered ", registered_model.id)
    return registered_model.id

model_name_id_mapping = dict()
for name, model in models_dominant.items():
    m_id = register_model(name, model)
    model_name_id_mapping[name] = m_id

Registering  census_unmitigated
Registering model census_unmitigated
Registered  census_unmitigated:2
Registering  census_gs_model_34
Registering model census_gs_model_34
Registered  census_gs_model_34:2
Registering  census_gs_model_35
Registering model census_gs_model_35
Registered  census_gs_model_35:2
Registering  census_gs_model_36
Registering model census_gs_model_36
Registered  census_gs_model_36:2
Registering  census_gs_model_37
Registering model census_gs_model_37
Registered  census_gs_model_37:2
Registering  census_gs_model_38
Registering model census_gs_model_38
Registered  census_gs_model_38:2
Registering  census_gs_model_39
Registering model census_gs_model_39
Registered  census_gs_model_39:2
Registering  census_gs_model_40
Registering model census_gs_model_40
Registered  census_gs_model_40:2
Registering  census_gs_model_41
Registering model census_gs_model_41
Registered  census_gs_model_41:2
Registering  census_gs_model_42
Registering model census_gs_model_42
Registered  c

Now, produce new predictions dictionaries, with the updated names:

In [17]:
predictions_dominant_ids = dict()
for name, y_pred in predictions_dominant.items():
    predictions_dominant_ids[model_name_id_mapping[name]] = y_pred

<a id="PrecomputeMetrics"></a>
### Precomputing Metrics

We create a _dashboard dictionary_ using Fairlearn's `metrics` package. The `_create_group_metric_set` method has arguments similar to the Dashboard constructor, except that the sensitive features are passed as a dictionary (to ensure that names are available), and we must specify the type of prediction. Note that we use the `predictions_dominant_ids` dictionary we just created:

In [18]:
sf = {'race': A_test.race }

from fairlearn.metrics._group_metric_set import _create_group_metric_set


dash_dict = _create_group_metric_set(y_true=y_test,
                                     predictions=predictions_dominant_ids,
                                     sensitive_features=sf,
                                     prediction_type='binary_classification')

<a id="DashboardUpload"></a>
### Uploading the Dashboard

Now, we import our `contrib` package which contains the routine to perform the upload:

In [19]:
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

Now we can create an Experiment, then a Run, and upload our dashboard to it:

In [20]:
exp = Experiment(ws, "Fairlearn_GridSearch_Experiment_for_Race_v2")
print(exp)

run = exp.start_logging()
try:
    dashboard_title = "Dominant Models from GridSearch"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
finally:
    run.complete()

Experiment(Name: Fairlearn_GridSearch_Experiment_for_Race_v2,
Workspace: amlws-sufse2t562jqy-pm0508)

Uploaded to id: 68f28752-ce7a-4552-a50b-f4f92b993900



INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_dashboard_validation.py:Starting validation of dashboard dictionary
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_dashboard_validation.py:Validation of dashboard dictionary successful
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Validating model ids exist
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Checking census_gs_model_34:2
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Checking census_gs_model_35:2
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Checking census_gs_model_36:2
INFO:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/contrib/fairness/_azureml_validation.py:Checking census_gs_model_3

The dashboard can be viewed in the Run Details page.

Finally, we can verify that the dashboard dictionary which we downloaded matches our upload:

In [21]:
print(dash_dict == downloaded_dict)

True


<a id="Conclusion"></a>
## Conclusion

In this notebook we have demonstrated how to use the `GridSearch` algorithm from Fairlearn to generate a collection of models, and then present them in the fairness dashboard in Azure Machine Learning Studio. Please remember that this notebook has not attempted to discuss the many considerations which should be part of any approach to unfairness mitigation. The [Fairlearn website](http://fairlearn.org/) provides that discussion